# Building databases of published works  

> Pragmatic tools for constructing databases of scientific works based on queries defined with Boolean Logic.

In [1]:
#| default_exp utils.db

In [2]:
#| hide
from nbdev import *

Tabulate queries in a spreadsheet and generate a database based on the data from those queries. 

**Example**:  Define a dataframe with an `id` column and a `query` column (expressing a search query in Boolean Logic):

| ID | DISEASE NAME | QUERY  | 
|----|--------------|--------|
| 1 | Adult Polyglucosan Body Disease | adult polyglucosan body disease \| adult polyglucosan body neuropathy
| 2 | AGAT deficiency |  "GATM deficiency" \| "AGAT deficiency" \| "arginine:glycine amidinotransferase deficiency" \| "L-arginine:glycine amidinotransferase deficiency"
| 3 | Guanidinoacetate methyltransferase deficiency | "guanidinoacetate methyltransferase deficiency" \| "GAMT deficiency"
| 4 | CLOVES Syndrome | "CLOVES syndrome \| (congenital lipomatous overgrowth) & (vascular malformation epidermal) & (nevi-spinal) & syndrome \| (congenital lipomatous overgrowth) & (vascular malformations) & (Epidermal nevi) & ((skeletal\|spinal) & abnormalities) \| CLOVE syndrome \| (congenital lipomatous overgrowth) & (vascular malformation) & (epidermal nevi)


In [1]:
#| export

import local_resources.linkml as linkml

from alhazen.utils.airtableUtils import AirtableUtils
from alhazen.utils.searchEngineUtils import ESearchQuery, EuroPMCQuery
from alhazen.utils.queryTranslator import QueryTranslator, QueryType
from alhazen.schema_sqla import ScientificPublication, ScientificPublicationCollection
import alhazen.schema_python as linkml_py
from alhazen.utils.jats_text_extractor import NxmlDoc
from alhazen.utils.ceifns_db import *

from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.schema.runnable import RunnableLambda

from bs4 import BeautifulSoup,Tag,Comment,NavigableString
from databricks import sql
from datetime import datetime
from importlib_resources import files
import os
import pandas as pd
from pathlib import Path
import re
import requests
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker
from time import time,sleep
from tqdm import tqdm
from urllib.request import urlopen
from urllib.parse import quote_plus, quote, unquote
from urllib.error import URLError, HTTPError

In [2]:
db = Ceifns_LiteratureDb('/Users/gburns/alhazen/', 'em_tech' )
if db.session is None:
    session_class = sessionmaker(bind=db.engine)
    db.session = session_class()

In [3]:
from io import StringIO

EM_QUERIES_TSV = '''
ID,NAME,QUERY
0,Hierarchical phase-contrast tomography,Hierarchical phase-contrast tomography | HIP-CT | Hierarchical phase contrast tomography
1,Cryo-Electron Tomography,Cryoelectron Tomography | Cryo Electron Tomography | Cryo-Electron Tomography | Cryo-ET | CryoET
2,Volume Electron Microscopy,Volume Electron Microscopy | Volume EM | (serial section & (electron microscopy | EM | transmission electron microscopy | TEM | scanning electron microscopy | SEM | electron tomography )) | (serial block-face & (SEM | scanning electron microscopy)) | (focused ion beam & (SEM | scanning electron microscopy)) | (automated serial & (TEM | transmission electron microscopy)) | ( massively parallel imaging & (SEM | scanning electron microscopy)) | multibeam SEM | FAST-SEM | cryo-TEM
'''
EM_QUERIES_TSV = '''
ID,NAME,QUERY
0,Cryo-Electron Tomography,Cryoelectron Tomography | Cryo Electron Tomography | Cryo-Electron Tomography | Cryo-ET | CryoET
'''

cdf = pd.read_csv(StringIO(EM_QUERIES_TSV), sep=',')
qs = QuerySpec('EM Technology', 'ID', 'QUERY', 'NAME', {}, ['TITLE','ABSTRACT', 'METHODS'])
qt = QueryTranslator(cdf.sort_values('ID'), 'ID', 'QUERY', 'NAME')

db.add_corpus_from_epmc(qt, None, sections=qs.sections)


100%|██████████| 1/1 [00:00<00:00, 864.27it/s]


100%|██████████| 1/1 [00:00<00:00, 1506.57it/s]


https://www.ebi.ac.uk/europepmc/webservices/rest/search?format=JSON&pageSize=1000&synonym=TRUE&resultType=core&query=((TITLE:"Cryoelectron Tomography" OR ABSTRACT:"Cryoelectron Tomography" OR METHODS:"Cryoelectron Tomography") OR (TITLE:"Cryo Electron Tomography" OR ABSTRACT:"Cryo Electron Tomography" OR METHODS:"Cryo Electron Tomography") OR (TITLE:"Cryo-Electron Tomography" OR ABSTRACT:"Cryo-Electron Tomography" OR METHODS:"Cryo-Electron Tomography") OR (TITLE:"Cryo-ET" OR ABSTRACT:"Cryo-ET" OR METHODS:"Cryo-ET") OR (TITLE:"CryoET" OR ABSTRACT:"CryoET" OR METHODS:"CryoET")), 2446 European PMC PAPERS FOUND


  0%|          | 0/3 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [3]:
list_c = db.session.query(ScientificPublicationCollection).all()
for c in list_c:
  print('ID: %s | Name: %s'%(c.id,c.name))  

ID: 0 | Name: Cryo-Electron Tomography


In [5]:
pdf_path = '/tmp/alhazen/pdf_files/'
nxml_path = '/tmp/alhazen/nxml_files/'

l = []
for p in tqdm(db.list_corpus_publications('0')):

    has_full_text = False
    has_full_text_methods = False
    has_pdf = False
    
    pdf_file_path = pdf_path+p.doi+'.pdf'
    if os.path.exists(pdf_file_path):
        has_pdf = True

    nxml_file_path = nxml_path+p.doi+'.nxml'
    if os.path.exists(nxml_file_path):
        with open(nxml_file_path, 'r') as f:
            xml = f.read()
            soup = BeautifulSoup(xml, "lxml")
        body = soup.find_all('body')
        if body:
            has_full_text = True
        d = NxmlDoc(p.doi, xml)
        m = '\n'.join([d.read_section_text(sec) for sec in d.search_section_titles('methods')])
        if(len(m) > 0):
            has_full_text_methods = True

    l.append({'id':p.id, 'title':p.title, 'doi':p.doi, 'pub_date':p.publication_date, 'has_full_text':has_full_text, 'has_full_text_methods':has_full_text_methods, 'has_pdf':has_pdf})

df = pd.DataFrame(l)
df        

0it [00:00, ?it/s]


AttributeError: 'NoneType' object has no attribute 'has_part'

In [41]:
# query a specific paper and run an analysis on that paper.
#p = db.session.query(ScientificPublication) \
#            .filter(ScientificPublication.doi=='10.1038/s41592-022-01746-2').first()
p = db.session.query(ScientificPublication) \
            .filter(ScientificPublication.doi=='10.1038/s41586-022-05255-2').first()


In [42]:
print(p.doi)
print(p.title)
print(p.abstract)
print(p.publication_date)
print('~~~~~~~~~~~~~~~~~~~~~~~~~~')


10.1038/s41586-022-05255-2
Visualizing translation dynamics at atomic detail inside a bacterial cell.
Translation is the fundamental process of protein synthesis and is catalysed by the ribosome in all living cells<sup>1</sup>. Here we use advances in cryo-electron tomography and sub-tomogram analysis<sup>2,3</sup> to visualize the structural dynamics of translation inside the bacterium Mycoplasma pneumoniae. To interpret the functional states in detail, we first obtain a high-resolution in-cell average map of all translating ribosomes and build an atomic model for the M. pneumoniae ribosome that reveals distinct extensions of ribosomal proteins. Classification then resolves 13 ribosome states that differ in their conformation and composition. These recapitulate major states that were previously resolved in vitro, and reflect intermediates during active translation. On the basis of these states, we animate translation elongation inside native cells and show how antibiotics reshape the 

In [5]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 0)

In [43]:
path = db.loc+db.name+'/nxml_files/'
nxml_path = path+p.doi+'.nxml'
print(nxml_path)
if os.path.exists(nxml_path):
    with open(nxml_path, 'r') as f:
        xml = f.read()
        d = NxmlDoc(p.doi, xml)
        #sections_df = pd.DataFrame([(t.start, t.end, d.text[t.start+1:t.end]) for t in d.standoffs if t.element.tag=='sec'], 
        #                           columns=['start', 'end', 'text'])
        #sections_df['first_line']= sections_df.text.apply(lambda x: x.split('\n')[0])   
        #methods_df = sections_df[sections_df.first_line.str.contains('method', case=False)]
        #methods_text = methods_df.text.str.cat(sep='\n')
    
        df2 = d.build_simple_document_dataframe()
df2

/Users/gburns/alhazen/em_tech/nxml_files/10.1038/s41586-022-05255-2.nxml


,PMID,PARAGRAPH_ID,TAG,TOP_SECTION,SECTION,OFFSET,LENGTH,FIG_REF,PLAIN_TEXT
0,10.1038/s41586-022-05255-2,0,article-title,,,153,73,,Visualizing translation dynamics at atomic det...
1,10.1038/s41586-022-05255-2,1,abstract,,,2557,1458,,\nTranslation is the fundamental process of pr...
2,10.1038/s41586-022-05255-2,2,abstract,,,4016,252,,\nCryo-electron tomography is used to reveal t...
3,10.1038/s41586-022-05255-2,3,title,Main,Main,4480,4,,Main
4,10.1038/s41586-022-05255-2,4,p,Main,Main,4485,2002,,Translation of genetic information through mes...
...,...,...,...,...,...,...,...,...,...
139,10.1038/s41586-022-05255-2,139,title,,,70155,17,,Code availability
140,10.1038/s41586-022-05255-2,140,p,,,70173,414,,The code and associated data for bioinformatic...
141,10.1038/s41586-022-05255-2,141,title,,,70590,19,,Competing interests
142,10.1038/s41586-022-05255-2,142,p,,,70610,43,,The authors declare no competing interests.


In [46]:
df3 = df2[df2.TOP_SECTION.str.contains('method', case=False)]
df4 = df3.groupby('SECTION').agg({'OFFSET': 'min', 'LENGTH': 'sum', 'PLAIN_TEXT':lambda x: '\n'.join(x)}).sort_values('OFFSET')
df4.reset_index(drop=False)

,SECTION,OFFSET,LENGTH,PLAIN_TEXT
0,Methods,29117,7,Methods
1,Cryo-ET sample preparation and data collection,29126,1332,Cryo-ET sample preparation and data collection...
2,"Image processing, ribosome template matching a...",30464,1304,"Image processing, ribosome template matching a..."
3,Atomic model building in high-resolution ribos...,31773,1593,Atomic model building in high-resolution ribos...
4,Bioinformatic analysis of ribosomal proteins,33370,6783,Bioinformatic analysis of ribosomal proteins\n...
5,Sub-tomogram classification of the translation...,40161,4106,Sub-tomogram classification of the translation...
6,Model building and comparison of the ribosome ...,44275,1844,Model building and comparison of the ribosome ...
7,Spatial analysis of ribosomes and polysomes,46124,3155,Spatial analysis of ribosomes and polysomes\nS...
8,Statistical analysis of translation elongation...,49286,3227,Statistical analysis of translation elongation...
9,Single-cell clustering analysis,52519,858,Single-cell clustering analysis\nThe distribut...


In [47]:
# A global registry for all prompt templates
#global instructions
from alhazen.core import PromptTemplateRegistry

instructions = PromptTemplateRegistry()
instructions.load_prompts_from_yaml('ft_method_metadata_extraction.yaml')
pt = instructions.get_instruction_template('cryoet metadata extraction').generate_llama2_prompt_template()
pt

PromptTemplate(input_variables=['metadata_name', 'metadata_specification', 'section_text'], template="<s>[INST]\n                <<SYS>>You are an expert biological scientist trained in imaging and microscopy.<</SYS>>\n                Read the text of a section from the methods section of a research paper shown below (delimited with triple backticks).\nSection Text:- '''{section_text}'''\nExtract the metadata from the section text based only on the specification delineated by square brackets and record the answer in the 'metadata_value' field of the output. Do not extract any other metadata.\n[{metadata_specification}]\nRecord the term '{metadata_name}' in the 'metadata_name' field of the output.\nGenerate only JSON formatted output with three fields 'metadata_name', 'metadata_value' and 'orginal_text'. Do not provide additional explanation or context for the answer.\nIf the metadata is not present in the text, record a value of 'not present' in the 'metadata_value' field of the output

In [48]:
from functools import partial

from alhazen.core import PromptTemplateRegistry, get_langchain_llm, get_cached_gguf, get_langchain_embeddings, GGUF_LOOKUP_URL, MODEL_TYPE

from langchain.callbacks.manager import CallbackManagerForChainRun
from langchain.chains.combine_documents import collapse_docs, split_list_of_docs
from langchain.llms import LlamaCpp 
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough


In [49]:
os.environ['LLMS_TEMP_DIR'] = '/Users/gburns/alhazen/'
n_gpu_layers = 1 #kwargs.get('n_gpu_layers', 1)
temperature = 0.1 #kwargs.get('temperature', 0.1)
n_batch = 512 #kwargs.get('n_batch', 512)  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
model_path = get_cached_gguf('llama-2-70b-chat')
n_ctx = 4096 #kwargs.get('n_ctx', 4096)

llm = LlamaCpp(
    model_path=model_path,
    n_ctx=n_ctx,
    n_gpu_layers=n_gpu_layers,
    temperature=temperature,
    n_batch=n_batch,
    f16_kv=True,
    verbose=True, # Verbose is required to pass to the callback manager
)    

#llm = get_langchain_llm('llama-2-70b-chat')   


llama_model_loader: loaded meta data with 19 key-value pairs and 723 tensors from /Users/gburns/alhazen/llama-2-70b-chat.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  8192, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 28672,  8192,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  8192,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  8192

In [50]:
from alhazen.utils.langchain_utils import suppress_stdout_stderr, JsonEnclosedByTextOutputParser
from langchain.schema import OutputParserException

instructions = PromptTemplateRegistry()
instructions.load_prompts_from_yaml('ft_method_metadata_extraction.yaml')
pt1 = instructions.get_instruction_template('cryoet protocol step identification').generate_llama2_prompt_template()

pt2 = instructions.get_instruction_template('cryoet metadata extraction').generate_llama2_prompt_template()
metadata_specs = [
    {'step': 'AB', 'name': 'biological_sample_type', 'spec': 'What is the type of biological sample being used? Select from the following list: cell culture, tissue, organoid, organ, whole organism, virus, micro-organism, other.'},
    {'step': 'AB', 'name': 'organism_name', 'spec': 'If the biological sample was taken from an organism, provide the scientific name of the organism. If the biological sample is not from an organism, return none.'},
    {'step': 'AB', 'name': 'cell_strain', 'spec': 'If the biological sample described in the text is a cell or cell culture, what was the strain?'},
    {'step': 'AB', 'name': 'sample_preparation', 'spec': 'Provide a summary of any actions performed on the biological sample.'},
    {'step': 'C', 'name': 'grid_preparation', 'spec': 'Provide a summary of how grids were prepared for electron microscopy.'},
    {'step': 'D', 'name': 'cryoet_pixel_spacing', 'spec': ' What was the pixel spacing the CryoET step?'},
    {'step': 'D', 'name': 'cryoet_acceleration_voltage', 'spec': 'What was the acceleration voltage?'},
    {'step': 'D', 'name': 'microscope_name', 'spec': 'What was the name of the type of electron microscope being used?'},
    {'step': 'D', 'name': 'microscope_setup', 'spec': 'Which, if any, of the following electron microscopy methods were used:- (A) an energy filter, (B) a phase plate, or (C) an image corrector?'},
    {'step': 'D', 'name': 'camera_manufacturer', 'spec': 'Which company made the camera used to capture tilt images?'},
    {'step': 'D', 'name': 'camera_model', 'spec': 'What model of camera was used to capture tilt images?'},
    {'step': 'D', 'name': 'tilt_minimum', 'spec': 'What was the minimum angle used in the tilt-series?'},
    {'step': 'D', 'name': 'tilt_maximum', 'spec': 'What was the maximum angle used in the tilt-series?'},
    {'step': 'D', 'name': 'total_flux', 'spec': 'What was the total flux or total exposure dose?'},
    {'step': 'E', 'name': 'reconstruction_software', 'spec': 'What software or computational methods were used to perform the tomogram reconstruction?'},
    {'step': 'E', 'name': 'tomogram_size', 'spec': 'What are the dimensions of the tomogram dataset (measured in numbers of pixels)?'},
    {'step': 'E', 'name': 'reconstruction_method', 'spec': 'What computational methods / algorithms were used (e.g., weighted back propagation, etc)?'},
    {'step': 'F', 'name': 'annotated_entities', 'spec': 'What organelles or subcellular components were annotated in the images?'},
    {'step': 'F', 'name': 'annotation_methods', 'spec': 'List all the methods describing how annotations were generated (manually, computational analysis, machine learning, etc)?'},
    {'step': 'F', 'name': 'annotated_software', 'spec': 'What software was used to make the annotations?'},
] 
docs = [ row.PLAIN_TEXT 
    for i, row in df4.iterrows()
    if row.LENGTH > 50
]

# Can't use callbacks with LCEL constructs
protocol_step_id_lcel = pt1 | llm | JsonEnclosedByTextOutputParser()
extract_lcel = pt2 | llm | JsonEnclosedByTextOutputParser()

extractions = []
for i, row in df4.reset_index(drop=False).iterrows():
    print('\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print(i, row.OFFSET, row.PLAIN_TEXT)
    if row.LENGTH <= 50:
        continue
    #if i<11:
    #    continue
    s1 = {'section_text':row.PLAIN_TEXT }
    
    protocol_step = None
    attempts = 0
    while protocol_step is None and attempts < 5:
        try: 
            #with suppress_stdout_stderr():
            out1 = protocol_step_id_lcel.invoke(s1, config={'callbacks': [ConsoleCallbackHandler()]})
            if out1 is not None:
                protocol_step = out1.get('protocol_step', None)
            else: 
                protocol_step = 'X'
        except OutputParserException as e:
            attempts += 1
            print(e) 
            continue

    print('\t'+protocol_step)  
    for spec in metadata_specs:
        print('\t'+spec.get('name'))
        if protocol_step not in spec.get('step') :
            continue
        s2 = {'section_text':row.PLAIN_TEXT, 'metadata_specification': spec.get('spec'), 'metadata_name': spec.get('name') }
        
        try:
            #with suppress_stdout_stderr():
            out2 = extract_lcel.invoke(s2, config={'callbacks': [ConsoleCallbackHandler()]})
        except OutputParserException as e:
            continue
        if out2 is not None:
            out2['p_id'] = i
            out2['offset'] = row.OFFSET
            out2['length'] = row.LENGTH
            extractions.append(out2)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')


ggml_metal_free: deallocating




~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0 29117 Methods


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 29126 Cryo-ET sample preparation and data collection
The M. pneumoniae cultivation, sample preparation and data collection were described previously2. The three datasets of native untreated, Cm-treated and PUM-treated cells were re-processed in Warp and M 1.0.7 (the alpha versions that were officially released as v.1.0.9.)3,54. A small dataset of 15 tomograms acquired with Volta phase plate was processed with the denoising network in Warp 1.0.9 for visualization purposes only (as shown in Fig. 1a).
A dataset of spectinomycin-treated cells was collected following the same procedure as before2. In brief, spectinomycin (Sigma-Aldrich) at a final concentration of 0.4 mg ml-1 was added into the culture medium, 15-20 min before plunge-freezing. Tilt-series collection with the dose-symmetric scheme55 was performed on a Titan Krios transmission electron microscope equipped with a K3 camera (


llama_print_timings:        load time = 10455.37 ms
llama_print_timings:      sample time =   122.00 ms /   110 runs   (    1.11 ms per token,   901.66 tokens per second)
llama_print_timings: prompt eval time = 17214.07 ms /   837 tokens (   20.57 ms per token,    48.62 tokens per second)
llama_print_timings:        eval time = 19574.83 ms /   109 runs   (  179.59 ms per token,     5.57 tokens per second)
llama_print_timings:       total time = 37195.23 ms
Llama.generate: prefix-match hit


[llm/end] [1:chain:RunnableSequence > 3:llm:LlamaCpp] [37.20s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "  {\n\"protocol_step\": \"A\"\n}\n\nThe text describes the preparation of a biological sample for imaging, specifically the cultivation and sample preparation of M. pneumoniae cells, as well as the addition of various treatments (Cm-treated, PUM-treated, and Spc-treated) to the cells before plunge-freezing. Therefore, option A (the preparation of a biological sample for imaging) is the most appropriate choice.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 4:parser:JsonEnclosedByTextOutputParser] Entering Parser run with input:
{
  "input": "  {\n\"protocol_step\": \"A\"\n}\n\nThe text describes the preparation of a biological sample for imaging, specifically the cultivation and sample preparation of M. pneumoniae cells, as well a


llama_print_timings:        load time = 10455.37 ms
llama_print_timings:      sample time =    41.15 ms /    57 runs   (    0.72 ms per token,  1385.08 tokens per second)
llama_print_timings: prompt eval time = 14021.93 ms /   729 tokens (   19.23 ms per token,    51.99 tokens per second)
llama_print_timings:        eval time = 10065.44 ms /    56 runs   (  179.74 ms per token,     5.56 tokens per second)
llama_print_timings:       total time = 24199.13 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time = 10455.37 ms
llama_print_timings:      sample time =    87.33 ms /   118 runs   (    0.74 ms per token,  1351.18 tokens per second)
llama_print_timings: prompt eval time =  5086.36 ms /   233 tokens (   21.83 ms per token,    45.81 tokens per second)
llama_print_timings:        eval time = 21194.94 ms /   117 runs   (  181.15 ms per token,     5.52 tokens per second)
llama_print_timings:       total time = 26520.75 ms
Llama.generate: prefix-match hit


...


llama_print_timings:        load time = 10455.37 ms
llama_print_timings:      sample time =    82.44 ms /   114 runs   (    0.72 ms per token,  1382.79 tokens per second)
llama_print_timings: prompt eval time =  4759.82 ms /   211 tokens (   22.56 ms per token,    44.33 tokens per second)
llama_print_timings:        eval time = 22544.77 ms /   113 runs   (  199.51 ms per token,     5.01 tokens per second)
llama_print_timings:       total time = 27529.15 ms
Llama.generate: prefix-match hit


[llm/end] [1:chain:RunnableSequence > 3:llm:LlamaCpp] [27.53s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "  {\n\"metadata_name\": \"tilt_minimum\",\n\"metadata_value\": \"not present\",\n\"original_text\": \"Any methods, additional references, Nature Research reporting summaries, source data, extended data, supplementary information, acknowledgements, peer review information; details of author contributions and competing interests; and statements of data and code availability are available at 10.1038/s41586-022-05255-2.\"\n}",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 4:parser:JsonEnclosedByTextOutputParser] Entering Parser run with input:
{
  "input": "  {\n\"metadata_name\": \"tilt_minimum\",\n\"metadata_value\": \"not present\",\n\"original_text\": \"Any methods, additional references, Nature Research reporting summaries, source


llama_print_timings:        load time = 10455.37 ms
llama_print_timings:      sample time =    83.07 ms /   114 runs   (    0.73 ms per token,  1372.35 tokens per second)
llama_print_timings: prompt eval time =  4760.12 ms /   209 tokens (   22.78 ms per token,    43.91 tokens per second)
llama_print_timings:        eval time = 22545.53 ms /   113 runs   (  199.52 ms per token,     5.01 tokens per second)
llama_print_timings:       total time = 27528.74 ms
Llama.generate: prefix-match hit


[llm/end] [1:chain:RunnableSequence > 3:llm:LlamaCpp] [27.53s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "  {\n\"metadata_name\": \"tilt_maximum\",\n\"metadata_value\": \"not present\",\n\"original_text\": \"Any methods, additional references, Nature Research reporting summaries, source data, extended data, supplementary information, acknowledgements, peer review information; details of author contributions and competing interests; and statements of data and code availability are available at 10.1038/s41586-022-05255-2.\"\n}",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 4:parser:JsonEnclosedByTextOutputParser] Entering Parser run with input:
{
  "input": "  {\n\"metadata_name\": \"tilt_maximum\",\n\"metadata_value\": \"not present\",\n\"original_text\": \"Any methods, additional references, Nature Research reporting summaries, source


llama_print_timings:        load time = 10455.37 ms
llama_print_timings:      sample time =    81.34 ms /   113 runs   (    0.72 ms per token,  1389.30 tokens per second)
llama_print_timings: prompt eval time =  4816.11 ms /   207 tokens (   23.27 ms per token,    42.98 tokens per second)
llama_print_timings:        eval time = 22419.14 ms /   112 runs   (  200.17 ms per token,     5.00 tokens per second)
llama_print_timings:       total time = 27454.80 ms


In [51]:
path = db.loc+db.name+'/extractions/'
if not os.path.exists(path+p.doi):
    os.makedirs(path+p.doi)


In [52]:
extractions_df = pd.DataFrame(extractions)
fls = []
for i, row in extractions_df.iterrows():
    text = d.text[row.offset:row.offset+row.length]
    fls.append(text.split('\n')[0])
extractions_df['first_line'] = fls

In [53]:
extractions_pivot = extractions_df.pivot(index='metadata_name', columns=['offset', 'first_line'], values='metadata_value').fillna('')
extractions_pivot

offset,29126,30464,31773,40161,44275,46124,49286,52519,53382,53520
first_line,Cryo-ET sample preparation and data collection,"Image processing, ribosome template matching and map refinement",Atomic model building in high-resolution ribosome maps,Sub-tomogram classification of the translation states of ribosomes,Model building and comparison of the ribosome translation states,Spatial analysis of ribosomes and polysomes,Statistical analysis of translation elongation states in polysomes,Single-cell clustering analysis,Reporting summary,Online content
metadata_name,,,,,,,,,,
biological_sample_type,cell culture,,,,,,,,,
camera_manufacturer,,,,,,,,,not present,not present
camera_model,,,,,,,,,not present,not present
cell_strain,M. pneumoniae,,,,,,,,,
cryoet_acceleration_voltage,,,,,,,,,not present,not present
cryoet_pixel_spacing,,,,,,,,,not present,not present
microscope_name,,,,,,,,,not present,not present
microscope_setup,,,,,,,,,not present,not present


In [54]:
extractions_pivot.to_csv(path+'/'+p.doi+'/extractions.tsv', sep='\t', index=True, header=True)

In [40]:
extractions_pivot2 = extractions_df.pivot(index='metadata_name', columns=['offset', 'first_line'], values='original_text').fillna('')
extractions_pivot2.to_csv(path+'/'+p.doi+'/original_text.tsv', sep='\t', index=True, header=True)

In [28]:
text1 = '{\n\"protocol_step\": E\n}'
text2 = re.sub(r':\s*([a-zA-Z0-9_]+)\s*[,\]\}]', r': "\1"', text1)
print(text2)

{
"protocol_step": "E"


In [ ]:
for i, row in df4.reset_index(drop=False).iterrows():
    print(i, row.OFFSET, row.PLAIN_TEXT)
    if row.LENGTH <= 50:
        continue
    if i < 11:
        continue
    s1 = {'section_text':row.PLAIN_TEXT }
    with suppress_stdout_stderr():
        out1 = protocol_step_id_chain.invoke(s1)
    if out1:
        protocol_step = out1.get('protocol_step', None)
    else:
        continue
    if protocol_step is None:
        raise Exception('No protocol step found')      
    print('\t'+protocol_step)  
    for spec in metadata_specs:
        print('\t'+spec.get('name'))
        if protocol_step not in spec.get('step') :
            continue
        s2 = {'section_text':row.PLAIN_TEXT, 'metadata_specification': spec.get('spec'), 'metadata_name': spec.get('name') }
        try:
            with suppress_stdout_stderr():
                out2 = extract_chain.invoke(s2)
        except OutputParserException as e:
            continue
        if out2 is not None:
            out2['p_id'] = i
            out2['offset'] = row.OFFSET
            out2['length'] = row.LENGTH
            extractions.append(out2)

In [ ]:
import json
print(json.dumps(extractions, indent=4))

[
    {
        "metadata_name": "biological_sample_type",
        "metadata_value": "cell culture",
        "original_text": "S. pombe K972 Sp h- wild-type haploid cells were recovered from frozen stock by streaking on YES agar plates (YES Broth, Formedium, 20 g agarose per liter) and incubated at 30 degreesC for 1-3 days."
    },
    {
        "metadata_name": "organism_name",
        "metadata_value": "S. pombe K972 Sp h- wild-type haploid cells",
        "original_text": "S. pombe K972 Sp h- wild-type haploid cells were recovered from frozen stock by streaking on YES agar plates (YES Broth, Formedium, 20 g agarose per liter) and incubated at 30 degreesC for 1-3 days."
    },
    {
        "metadata_name": "cell_strain",
        "metadata_value": "S. pombe K972 Sp h- wild-type haploid",
        "original_text": "S. pombe K972 Sp h- wild-type haploid cells were recovered from frozen stock by streaking on YES agar plates (YES Broth, Formedium, 20 g agarose per liter) and incubated at 

In [59]:
import json
print(json.dumps(out, indent=4))

{
    "metadata_name": "biological_sample_type",
    "metadata_value": "cell culture",
    "original_text": "S. pombe K972 Sp h- wild-type haploid cells were recovered from frozen stock by streaking on YES agar plates (YES Broth, Formedium, 20 g agarose per liter) and incubated at 30 degreesC for 1-3 days."
}


In [ ]:
map_as_doc_chain.invoke({docs[0].page_content})

In [13]:
# The chain we'll repeatedly apply to collapse subsets of the documents
# into a consolidate document until the total token size of our
# documents is below some max size.
def format_docs(docs):
    return "\n\n".join(partial_format_document(doc) for doc in docs)

collapse_chain = (
    {"context": format_docs}
    | PromptTemplate.from_template("Collapse this content:\n\n{context}")
    | llm
    | StrOutputParser()
)

def get_num_tokens(docs):
    return llm.get_num_tokens(format_docs(docs))

def collapse(docs, config, token_max=1000,):
    collapse_ct = 1
    while get_num_tokens(docs) > token_max:
        config["run_name"] = f"Collapse {collapse_ct}"
        invoke = partial(collapse_chain.invoke, config=config)
        split_docs = split_list_of_docs(docs, get_num_tokens, token_max)
        docs = [collapse_docs(_docs, invoke) for _docs in split_docs]
        collapse_ct += 1
    return docs

In [6]:
# The chain we'll use to combine our individual document summaries
# (or summaries over subset of documents if we had to collapse the map results)
# into a final summary.

reduce_chain = (
    {"context": format_docs}
    | PromptTemplate.from_template("Combine these summaries:\n\n{context}")
    | llm
    | StrOutputParser()
).with_config(run_name="Reduce")

# The final full chain
map_reduce = (map_as_doc_chain.map() | collapse | reduce_chain).with_config(run_name="Map reduce")

In [8]:
from langchain.schema import Document

text = """Nuclear power in space is the use of nuclear power in outer space, typically either small fission systems or radioactive decay for electricity or heat. Another use is for scientific observation, as in a Mössbauer spectrometer. The most common type is a radioisotope thermoelectric generator, which has been used on many space probes and on crewed lunar missions. Small fission reactors for Earth observation satellites, such as the TOPAZ nuclear reactor, have also been flown.[1] A radioisotope heater unit is powered by radioactive decay and can keep components from becoming too cold to function, potentially over a span of decades.[2]

The United States tested the SNAP-10A nuclear reactor in space for 43 days in 1965,[3] with the next test of a nuclear reactor power system intended for space use occurring on 13 September 2012 with the Demonstration Using Flattop Fission (DUFF) test of the Kilopower reactor.[4]

After a ground-based test of the experimental 1965 Romashka reactor, which used uranium and direct thermoelectric conversion to electricity,[5] the USSR sent about 40 nuclear-electric satellites into space, mostly powered by the BES-5 reactor. The more powerful TOPAZ-II reactor produced 10 kilowatts of electricity.[3]

Examples of concepts that use nuclear power for space propulsion systems include the nuclear electric rocket (nuclear powered ion thruster(s)), the radioisotope rocket, and radioisotope electric propulsion (REP).[6] One of the more explored concepts is the nuclear thermal rocket, which was ground tested in the NERVA program. Nuclear pulse propulsion was the subject of Project Orion.[7]

Regulation and hazard prevention[edit]
After the ban of nuclear weapons in space by the Outer Space Treaty in 1967, nuclear power has been discussed at least since 1972 as a sensitive issue by states.[8] Particularly its potential hazards to Earth's environment and thus also humans has prompted states to adopt in the U.N. General Assembly the Principles Relevant to the Use of Nuclear Power Sources in Outer Space (1992), particularly introducing safety principles for launches and to manage their traffic.[8]

Benefits

Both the Viking 1 and Viking 2 landers used RTGs for power on the surface of Mars. (Viking launch vehicle pictured)
While solar power is much more commonly used, nuclear power can offer advantages in some areas. Solar cells, although efficient, can only supply energy to spacecraft in orbits where the solar flux is sufficiently high, such as low Earth orbit and interplanetary destinations close enough to the Sun. Unlike solar cells, nuclear power systems function independently of sunlight, which is necessary for deep space exploration. Nuclear-based systems can have less mass than solar cells of equivalent power, allowing more compact spacecraft that are easier to orient and direct in space. In the case of crewed spaceflight, nuclear power concepts that can power both life support and propulsion systems may reduce both cost and flight time.[9]

Selected applications and/or technologies for space include:

Radioisotope thermoelectric generator
Radioisotope heater unit
Radioisotope piezoelectric generator
Radioisotope rocket
Nuclear thermal rocket
Nuclear pulse propulsion
Nuclear electric rocket
"""

docs = [
    Document(
        page_content=split,
        metadata={"source": "https://en.wikipedia.org/wiki/Nuclear_power_in_space"},
    )
    for split in text.split("\n\n")
]
docs

[Document(page_content='Nuclear power in space is the use of nuclear power in outer space, typically either small fission systems or radioactive decay for electricity or heat. Another use is for scientific observation, as in a Mössbauer spectrometer. The most common type is a radioisotope thermoelectric generator, which has been used on many space probes and on crewed lunar missions. Small fission reactors for Earth observation satellites, such as the TOPAZ nuclear reactor, have also been flown.[1] A radioisotope heater unit is powered by radioactive decay and can keep components from becoming too cold to function, potentially over a span of decades.[2]', metadata={'source': 'https://en.wikipedia.org/wiki/Nuclear_power_in_space'}),
 Document(page_content='The United States tested the SNAP-10A nuclear reactor in space for 43 days in 1965,[3] with the next test of a nuclear reactor power system intended for space use occurring on 13 September 2012 with the Demonstration Using Flattop Fis

In [14]:
map_reduce.invoke(docs, config={"max_concurrency": 1})


Llama.generate: prefix-match hit




The advantages of using nuclear power in space include:

* Long-lasting energy source that does not require refueling
* High energy density, which means smaller and lighter systems compared to other power sources
* Ability to operate in extreme temperatures and radiation environments
* Can be used for both electricity and heat
* Can be used for propulsion, such as with a nuclear-powered ion thruster

The disadvantages of using nuclear power in space include:

* Safety concerns, including the risk of radioactive contamination and accidents
* High cost of development and launch
* Potential for misuse, such as using it for weapons
* Requires specialized infrastructure and handling procedures
* Can have negative public perception

There are several challenges associated with using nuclear power in space, including:

* Radiation protection for the crew and electronics
* Heat rejection and temperature control
* Launch and deployment
* Operations and maintenance
* Decommissioning and dispos


llama_print_timings:        load time =  4949.59 ms
llama_print_timings:      sample time =   189.82 ms /   256 runs   (    0.74 ms per token,  1348.65 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 48455.88 ms /   256 runs   (  189.28 ms per token,     5.28 tokens per second)
llama_print_timings:       total time = 49102.73 ms
Llama.generate: prefix-match hit


 The DUFF test was conducted at the Nevada National Security Site's Device Assembly Facility, and it successfully demonstrated the ability to generate electricity from a fission reaction in space.

The Kilopower reactor is designed to provide 1-10 kilowatts of power for space missions lasting several years, and it uses a combination of nuclear fission and radioisotope thermoelectric generators (RTGs) to generate electricity. The reactor is also designed to be scalable, allowing it to be used in a variety of different configurations depending on the specific mission requirements.

The successful test of the Kilopower reactor represents a significant milestone for space nuclear power, as it demonstrates the feasibility of using fission reactors in space and paves the way for future missions that require a reliable source of power.

In addition to the DUFF test, NASA is also planning to conduct further tests of the Kilopower reactor in the coming years, including a long-duration test on t


llama_print_timings:        load time =  4949.59 ms
llama_print_timings:      sample time =   186.46 ms /   256 runs   (    0.73 ms per token,  1372.94 tokens per second)
llama_print_timings: prompt eval time =  1984.33 ms /    84 tokens (   23.62 ms per token,    42.33 tokens per second)
llama_print_timings:        eval time = 49690.56 ms /   255 runs   (  194.86 ms per token,     5.13 tokens per second)
llama_print_timings:       total time = 52310.30 ms
Llama.generate: prefix-match hit


...

Nuclear-powered ion thruster
Nuclear-powered Hall effect thruster

Classify the content into categories.

Please provide detailed information about each category and its subcategories, along with examples of each. Also, please explain the relationship between the categories.

It would be best if you could provide a diagram or flowchart to illustrate the relationships between the categories.

I'll be happy to help you with that!

The given content can be classified into the following categories:

1. Radioisotope-based power sources
	* Radioisotope thermoelectric generator (RTG)
	* Radioisotope heater unit (RHU)
	* Radioisotope piezoelectric generator (RPG)
2. Nuclear-based propulsion systems
	* Nuclear thermal rocket (NTR)
	* Nuclear pulse propulsion (NPP)
	* Nuclear electric rocket (NER)
3. Nuclear-powered electrical thrusters
	* Nuclear-powered ion thruster (NIT)
	* Nuclear-powered


llama_print_timings:        load time =  4949.59 ms
llama_print_timings:      sample time =   192.31 ms /   256 runs   (    0.75 ms per token,  1331.17 tokens per second)
llama_print_timings: prompt eval time =  1670.95 ms /    57 tokens (   29.31 ms per token,    34.11 tokens per second)
llama_print_timings:        eval time = 53701.19 ms /   255 runs   (  210.59 ms per token,     4.75 tokens per second)
llama_print_timings:       total time = 56041.17 ms
Llama.generate: prefix-match hit


 Hall effect thruster (NHET)
4. Safety and regulatory considerations
	* Radiation protection for crew and electronics
	* Heat rejection and temperature control
	* Launch and deployment
	* Operations and maintenance
	* Decommissioning and disposal
5. Future developments and advancements
	* Advanced reactor designs (fission or fusion reactors)
	* Scalable reactor designs for various mission requirements
	* Long-duration tests on the International Space Station (ISS) and lunar surface demonstration missions
6. Historical background and past missions
	* First nuclear-powered satellite (SNAP-10A)
	* First nuclear-powered satellite with a thermoelectric converter (SNAP-19)
	* Soviet Union's first nuclear-powered satellite (Kosmos 217)
	* Radioisotope rocket and REP (radioisotope electric propulsion)

The relationship between the categories is as follows:

Radioisotope-based power sources are used to generate electricity, which can then be used to power nuclear-based


llama_print_timings:        load time =  4949.59 ms
llama_print_timings:      sample time =   240.07 ms /   256 runs   (    0.94 ms per token,  1066.37 tokens per second)
llama_print_timings: prompt eval time = 55764.83 ms /  2328 tokens (   23.95 ms per token,    41.75 tokens per second)
llama_print_timings:        eval time = 57927.55 ms /   255 runs   (  227.17 ms per token,     4.40 tokens per second)
llama_print_timings:       total time = 114587.14 ms


" Hall effect thruster (NHET)\n4. Safety and regulatory considerations\n\t* Radiation protection for crew and electronics\n\t* Heat rejection and temperature control\n\t* Launch and deployment\n\t* Operations and maintenance\n\t* Decommissioning and disposal\n5. Future developments and advancements\n\t* Advanced reactor designs (fission or fusion reactors)\n\t* Scalable reactor designs for various mission requirements\n\t* Long-duration tests on the International Space Station (ISS) and lunar surface demonstration missions\n6. Historical background and past missions\n\t* First nuclear-powered satellite (SNAP-10A)\n\t* First nuclear-powered satellite with a thermoelectric converter (SNAP-19)\n\t* Soviet Union's first nuclear-powered satellite (Kosmos 217)\n\t* Radioisotope rocket and REP (radioisotope electric propulsion)\n\nThe relationship between the categories is as follows:\n\nRadioisotope-based power sources are used to generate electricity, which can then be used to power nuclear